# Legal System Dynamics: Introduction

Welcome to the **Legal System Dynamics** computational framework!

This notebook provides an interactive introduction to:
- Constitutional Lock-in Index (CLI)
- Synthetic populations
- Hysteresis in legal systems

## Table of Contents
1. Understanding CLI
2. Creating Synthetic Agents
3. Simulating a Simple Reform
4. Visualizing Hysteresis

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../code')

from synthetic_populations.agent import Agent, AgentType, IntentionalityLevel

print("✓ Libraries loaded successfully")

## Part 1: Understanding CLI

The Constitutional Lock-in Index (CLI) measures institutional rigidity:

$$\text{CLI} = 1 - \frac{\text{Reforms Sustained}}{\text{Reforms Attempted}}$$

- **CLI = 0**: All reforms succeed (perfectly fluid)
- **CLI = 1**: All reforms fail (perfectly rigid)
- **Optimal CLI ≈ 0.40-0.50**: Balance stability and adaptability

In [ ]:
# Load CLI dataset
cli_data = pd.read_csv('../data/processed/cli_scores.csv')

# Display top 5 most rigid systems
print("Top 5 most rigid constitutional systems:")
print(cli_data.nlargest(5, 'cli_score')[['country', 'cli_score', 'n_reforms_attempted', 'n_reforms_sustained']])

# Plot CLI distribution
plt.figure(figsize=(10, 6))
plt.barh(cli_data['country'], cli_data['cli_score'], color='steelblue')
plt.xlabel('CLI Score', fontsize=12)
plt.ylabel('Country', fontsize=12)
plt.title('Constitutional Lock-in Index by Country', fontsize=14, fontweight='bold')
plt.axvline(x=0.5, color='red', linestyle='--', label='Optimal range boundary')
plt.legend()
plt.tight_layout()
plt.show()

## Part 2: Creating Synthetic Agents

Agents have:
- **Intentionality levels** (Dennett 0-3)
- **Beliefs** about norms (liberty vs control)
- **Types** (citizen, judge, legislator, etc.)

In [ ]:
# Create a simple agent
agent1 = Agent(
    agent_id=1,
    agent_type=AgentType.CITIZEN,
    initial_beliefs={"liberty": 0.7, "control": 0.3}
)

print(agent1)
print(f"Intentionality level: {agent1.intentionality_level.value}")

# Create a more sophisticated agent (judge)
judge = Agent(
    agent_id=2,
    agent_type=AgentType.JUDGE
)

print("\n", judge)
print(f"Intentionality level: {judge.intentionality_level.value}")
print(f"Can form common knowledge: {agent1.can_form_common_knowledge_with(judge)}")

## Part 3: Simulating a Simple Reform

Let's simulate a population responding to reform pressure.

In [ ]:
# Create small population
n_agents = 50
agents = []

for i in range(n_agents):
    # Mix of liberty and control supporters
    if np.random.random() < 0.6:  # 60% liberty supporters
        beliefs = {"liberty": np.random.uniform(0.6, 0.9), "control": np.random.uniform(0.1, 0.4)}
    else:
        beliefs = {"liberty": np.random.uniform(0.1, 0.4), "control": np.random.uniform(0.6, 0.9)}
    
    agent = Agent(agent_id=i, initial_beliefs=beliefs)
    agents.append(agent)

print(f"Created {n_agents} agents")

# Apply reform pressure
reform_pressure = 0.8
print(f"Applying reform pressure: {reform_pressure}")

supporters_before = sum(1 for a in agents if a.beliefs["liberty"] > 0.5)
print(f"Supporters before: {supporters_before}/{n_agents}")

# Update beliefs
for agent in agents:
    agent.update_belief("liberty", reform_pressure, update_rule="bayesian")

supporters_after = sum(1 for a in agents if a.beliefs["liberty"] > 0.5)
print(f"Supporters after: {supporters_after}/{n_agents}")
print(f"Δ Supporters: +{supporters_after - supporters_before}")

## Part 4: Visualizing Hysteresis

Now let's create a simple hysteresis loop.

In [ ]:
# Simulate full cycle
pressures = list(np.linspace(0, 1, 10)) + list(np.linspace(1, 0, 10))
flexibility = []

# Reset agents
for agent in agents:
    agent.beliefs = {"liberty": 0.5, "control": 0.5}

for pressure in pressures:
    for agent in agents:
        agent.update_belief("liberty", pressure, update_rule="bayesian")
    
    supporters = sum(1 for a in agents if a.beliefs["liberty"] > 0.55)
    flexibility.append(supporters / n_agents)

# Plot
plt.figure(figsize=(8, 6))
plt.plot(pressures, flexibility, 'o-', linewidth=2, markersize=6)
plt.xlabel('Reform Pressure', fontsize=12)
plt.ylabel('Support Rate', fontsize=12)
plt.title('Simple Hysteresis Loop', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✓ Tutorial completed!")
print("\nNext steps:")
print("- Run code/experiments/exp01_hysteresis_loops.py for full simulation")
print("- Explore data/processed/cli_scores.csv")
print("- Read papers/ for theoretical background")